In [1]:
from pathlib import Path
import yaml
import jax.numpy as jnp
import numpy as np
from qdax.core.gp.encoding import compute_encoding_function
from bbbqd.body.bodies import encode_body
from functools import partial
from bbbqd.brain.controllers import compute_controller_generation_fn
from bbbqd.wrappers import make_env
from bbbqd.behavior.behavior_descriptors import _compute_spectrum
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
folder = "../results/evo-body-5x5-me-s3_0"
extra_prefix = "r3_"
config = yaml.safe_load(Path(f"{folder}/config.yaml").read_text())
config["skip"] = 1
config["episode_length"] = 5000

# Define encoding function
program_encoding_fn = compute_encoding_function(config)

# Define function to return the proper controller
controller_creation_fn = compute_controller_generation_fn(config)

# Body encoding function
body_encoding_fn = partial(encode_body, make_connected=True)

In [3]:
def get_genotype(position: int) -> jnp.ndarray:
    # Load fitnesses and genotypes
    try:
        fitnesses = jnp.load(f"{folder}/{extra_prefix}scores.npy")
    except FileNotFoundError:
        fitnesses = jnp.load(f"{folder}/{extra_prefix}fitnesses.npy")
    genotypes = jnp.load(f"{folder}/{extra_prefix}genotypes.npy")

    # Find best
    return genotypes[jnp.argmax(fitnesses)].astype(int) if position is None else genotypes[position].astype(int)

In [4]:
def full_descriptors(genome: jnp.ndarray) -> jnp.ndarray:
    body_genome, controller_genome = jnp.split(genome, [config["grid_size"] ** 2])
    controller = controller_creation_fn(program_encoding_fn(controller_genome))
    body = body_encoding_fn(body_genome)

    env = make_env(config, body)
    cumulative_reward = 0
    obs = env.reset()
    descriptors = []
    for _ in range(config["episode_length"]):
        action = controller.compute_action(obs)
        obs, reward, done, info = env.step(action)
        descriptors.append(
            np.asarray([info["velocity"][1], info["angle"][0]])
        )
        cumulative_reward += reward
        if done:
            break

    env.close()

    return np.asarray(descriptors).transpose()

In [7]:
position_names = {
    749: "low-x+high-y",
    687: "high-x+high-y",
    200: "high-x+low-y",
    967: "low-x+low-y",
}
descriptors_list = []
spectra_list = []
for pos, name in position_names.items():
    descriptors = full_descriptors(get_genotype(pos))
    spectra = np.asarray([_compute_spectrum(i) for i in descriptors])
    print(descriptors.shape)
    descriptors_list.append(descriptors)
    spectra_list.append(spectra)

In [6]:
colors = ["r", "y", "g", "b"]
plt.subplot(1, 2, 1)
for i in range(4):
    plt.plot(descriptors_list[i][0], c=colors[i])
plt.title("y velocity")
plt.subplot(1, 2, 2)
for i in range(4):
    plt.plot(descriptors_list[i][1], c=colors[i])
plt.title("angle")
plt.show()

In [ ]:
plt.subplot(1, 2, 1)
for i in range(4):
    plt.plot(spectra_list[i][0], c=colors[i])
plt.title("y velocity sp")
plt.subplot(1, 2, 2)
for i in range(4):
    plt.plot(spectra_list[i][1], c=colors[i])
plt.title("angle sp")
plt.show()